In [1]:
import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense, Embedding, LSTM, SpatialDropout1D
from sklearn.model_selection import train_test_split
from keras.utils.np_utils import to_categorical
from keras.callbacks import EarlyStopping
from keras.layers import Dropout
import re
from nltk.corpus import stopwords
from nltk import word_tokenize
STOPWORDS = set(stopwords.words('english'))
from bs4 import BeautifulSoup
import plotly.graph_objs as go
#import chart_studio.plotly as py
import cufflinks
from IPython.core.interactiveshell import InteractiveShell
import plotly.figure_factory as ff
InteractiveShell.ast_node_interactivity = 'all'
from plotly.offline import iplot
cufflinks.go_offline()
cufflinks.set_config_file(world_readable=True, theme='pearl')
import torch

/opt/conda/lib/python3.7/site-packages/geopandas/_compat.py:115: UserWarning:

The Shapely GEOS version (3.9.1-CAPI-1.14.2) is incompatible with the GEOS version PyGEOS was compiled with (3.10.1-CAPI-1.16.0). Conversions between both will be slow.



In [2]:
from keras.callbacks import ModelCheckpoint

In [3]:
train_data = pd.read_csv('../input/emotion-analysis-in-tamil/Emotion_Analysis_Task_A_train.csv', sep = '\t', header = None)
val_data = pd.read_csv('../input/emotion-analysis-in-tamil/Emotion_Analysis_Task_A_dev.csv', sep = '\t', header = None)
test_data = pd.read_csv('../input/emotion-analysis-in-tamil/Emotion_Analysis_Task_A_test.csv', sep = '\t', header = None)


In [4]:
train_data

,0,1
0,Neutral,நாளைக்கு அரிசிக்கு இந்த நிலமை வந்தா 🙂
1,Anger,மானம் கேட்ட அன்புமணி
2,Neutral,தவறு இஸ்ரேல் இருக்காது இதை நான் கூறவில்லை ஹமாஸ...
3,Joy,கொங்கு நாட்டு சிங்கம் உன்மையும் நேர்மையும் உலை...
4,Neutral,இவர் யார்? ஒவ்வொரு வார்த்தையும் முன்னுக்கு பின...
...,...,...
14203,Trust,பெ மணியரசன் கூறுவதைஉணர்ந்து. செயலாற்றுவதேஇன்ற...
14204,Ambiguous,இன்னும் எத்தன நாள் வச்சி செய்வீங்க.
14205,Anticipation,அடுத்த ஏதோ தயார்பன்னிட்டான்
14206,Ambiguous,தமிழ் மற்றும் சமஸ்கிருதம்


In [5]:
# The maximum number of words to be used. (most frequent)
MAX_NB_WORDS = 64000
# Max number of words in each complaint.
MAX_SEQUENCE_LENGTH = 128
# This is fixed.
EMBEDDING_DIM = 100
tokenizer = Tokenizer(num_words=MAX_NB_WORDS, filters='!"#$%&()*+,-./:;<=>?@[\]^_`{|}~', lower=False)
tokenizer.fit_on_texts(train_data[1])
word_index = tokenizer.word_index
print('Found %s unique tokens.' % len(word_index))

Found 38749 unique tokens.


In [6]:
X_train = tokenizer.texts_to_sequences(train_data[1])
X_train = pad_sequences(X_train, maxlen=MAX_SEQUENCE_LENGTH)
print('Shape of data tensor:', X_train.shape)

X_val = tokenizer.texts_to_sequences(val_data[1])
X_val = pad_sequences(X_val, maxlen=MAX_SEQUENCE_LENGTH)
print('Shape of data tensor:', X_val.shape)

X_test = tokenizer.texts_to_sequences(test_data[1])
X_test = pad_sequences(X_test, maxlen=MAX_SEQUENCE_LENGTH)
print('Shape of data tensor:', X_test.shape)

Shape of data tensor: (14208, 128)
Shape of data tensor: (3552, 128)
Shape of data tensor: (4440, 128)


In [7]:
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 4453351996969603542
, name: "/device:GPU:0"
device_type: "GPU"
memory_limit: 15667560448
locality {
  bus_id: 1
  links {
  }
}
incarnation: 12838064473617350522
physical_device_desc: "device: 0, name: Tesla P100-PCIE-16GB, pci bus id: 0000:00:04.0, compute capability: 6.0"
]


2022-03-05 10:10:18.352094: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-03-05 10:10:18.402286: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-03-05 10:10:18.403402: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-03-05 10:10:18.404216: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA 

In [8]:
Y_train = pd.get_dummies(train_data[0]).values
print('Shape of label tensor:', Y_train.shape)

Y_val = pd.get_dummies(val_data[0]).values
print('Shape of label tensor:', Y_val.shape)

Shape of label tensor: (14208, 11)
Shape of label tensor: (3552, 11)


In [9]:
def get_predicted(preds):
    pred_flat = np.argmax(preds, axis=1).flatten()
    return pred_flat
from sklearn.metrics import f1_score, accuracy_score
# def f1(ytrue, ypred):
#     return f1_score(get_predicted(np.array(ytrue)), get_predicted(np.array(ypred)), average = 'weighted')

In [10]:
from keras import backend as K
def f1(true, pred): #shapes (batch, 4)

    #for metrics include these two lines, for loss, don't include them
    #these are meant to round 'pred' to exactly zeros and ones
    #predLabels = K.argmax(pred, axis=-1)
    #pred = K.one_hot(predLabels, 4) 


    ground_positives = K.sum(true, axis=0)       # = TP + FN
    pred_positives = K.sum(pred, axis=0)         # = TP + FP
    true_positives = K.sum(true * pred, axis=0)  # = TP
        #all with shape (4,)

    precision = (true_positives + K.epsilon()) / (pred_positives + K.epsilon()) 
    recall = (true_positives + K.epsilon()) / (ground_positives + K.epsilon()) 
        #both = 1 if ground_positives == 0 or pred_positives == 0
        #shape (4,)

    f1 = 2 * (precision * recall) / (precision + recall + K.epsilon())
        #not sure if this last epsilon is necessary
        #matematically not, but maybe to avoid computational instability
        #still with shape (4,)

    weighted_f1 = f1 * ground_positives / K.sum(ground_positives)
    weighted_f1 = K.sum(weighted_f1)


    return weighted_f1 #for metrics, return only 'weighted_f1

In [11]:
mc = ModelCheckpoint('model_tam_1.h5', monitor='val_f1', mode='max', save_best_only=True)
cb_list = mc

In [12]:
from tensorflow.keras import layers

In [13]:
model = Sequential()
model.add(Embedding(MAX_NB_WORDS, EMBEDDING_DIM, input_length=X_train.shape[1]))
model.add(SpatialDropout1D(0.2))
model.add(layers.Bidirectional(LSTM(100, dropout=0.2, recurrent_dropout=0.2)))
model.add(Dense(11, activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=[f1])
history = model.fit(X_train, Y_train, validation_data=(X_val, Y_val), epochs=4, batch_size=32, callbacks = cb_list)
# Final evaluation of the model
model_pred_train = model.predict(X_train)
model_pred_test = model.predict(X_val)
# print(classification_report(test_labels,model_pred_test))
#print('LSTM Recurrent Neural Network baseline: ' + str(roc_auc_score(Y_train, model_pred_train)))
#print('LSTM Recurrent Neural Network: ' + str(roc_auc_score(Y_val, model_pred_test)))

2022-03-05 10:10:23.630917: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-03-05 10:10:23.631761: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-03-05 10:10:23.632390: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-03-05 10:10:23.633254: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-03-05 10:10:23.633906: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from S

Epoch 1/4
444/444 [==============================] - 211s 467ms/step - loss: 1.9128 - f1: 0.2150 - val_loss: 1.7757 - val_f1: 0.2609
Epoch 2/4
444/444 [==============================] - 207s 467ms/step - loss: 1.4663 - f1: 0.3520 - val_loss: 1.8301 - val_f1: 0.2889
Epoch 3/4
444/444 [==============================] - 209s 471ms/step - loss: 0.8322 - f1: 0.5960 - val_loss: 2.2417 - val_f1: 0.3113
Epoch 4/4
444/444 [==============================] - 209s 470ms/step - loss: 0.4285 - f1: 0.7788 - val_loss: 2.6241 - val_f1: 0.3065


In [14]:

model_pred_train = model.predict(X_train)
model_pred_test = model.predict(X_val)
f1 = f1_score(get_predicted(Y_train), get_predicted(model_pred_train), average = 'weighted')
acc = accuracy_score(get_predicted(Y_train), get_predicted(model_pred_train))
print(f1, acc)

0.9383760548006914 0.9396818693693694


In [15]:
f1 = f1_score(get_predicted(Y_val), get_predicted(model_pred_test), average = 'macro')
f1

0.21000761550619174

In [16]:
acc = accuracy_score(get_predicted(Y_val), get_predicted(model_pred_test))
print(f1, acc)

0.21000761550619174 0.3400900900900901


In [18]:
from sklearn.metrics import classification_report
report = classification_report(get_predicted(Y_val), get_predicted(model_pred_test),output_dict=True)


In [19]:
pd.DataFrame(report).transpose()

,precision,recall,f1-score,support
0,0.336364,0.338673,0.337514,437.00000
1,0.149733,0.152174,0.150943,184.00000
2,0.139738,0.150235,0.144796,213.00000
3,0.134615,0.133333,0.133971,210.00000
4,0.000000,0.000000,0.000000,23.00000
5,0.427340,0.621864,0.506569,558.00000
6,0.272727,0.047619,0.081081,189.00000
7,0.409539,0.407529,0.408532,1222.00000
8,0.320000,0.251309,0.281525,191.00000
9,0.000000,0.000000,0.000000,53.00000
